In [1]:
try: 
    import sys
    import numpy as np
    import tifffile as tf
    import os
    from osgeo import gdal, osr
    import earthpy as et
    import earthpy.spatial as es
    import math
    import imagecodecs
    #print('Import OK.')
    
except ImportError as IE:
    print('Import not OK - ', IE )
    sys.exit('exit - bad import')

In [2]:
#https://gist.github.com/jkatagi/a1207eee32463efd06fb57676dcf86c8
def fGeoref(input_array, src_dataset_path, output_path):
        cols = input_array.shape[1]
        rows = input_array.shape[0]
        
        dataset = gdal.Open(src_dataset_path, gdal.GA_ReadOnly)
        originX, pixelWidth, b, originY, d, pixelHeight = dataset.GetGeoTransform() 

        driver = gdal.GetDriverByName('GTiff')
        band_num = 1
        GDT_dtype = gdal.GDT_Float32
        outRaster = driver.Create(output_path, cols, rows, band_num, GDT_dtype)
        outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
        outband = outRaster.GetRasterBand(band_num)
        outband.WriteArray(input_array)
        prj=dataset.GetProjection()
        outRasterSRS = osr.SpatialReference(wkt=prj)
        outRaster.SetProjection(outRasterSRS.ExportToWkt())
        
        return True


In [3]:
def fndvi(red_path, nir_path, out_folder, name = "_NDVI"):

    red= tf.imread(red_path)
    nir = tf.imread(nir_path)

    zero_exception = np.seterr(all = "ignore")
    red_array = np.array(red).astype(np.float32)
    nir_array = np.array(nir).astype(np.float32)
    
    result_ndvi = np.divide((nir_array-red_array),(red_array+nir_array))
      
    result_ndvi_path = os.path.join(out_folder, name + ".TIF") 

    #tf.imsave(result_ndvi_path, result_ndvi)
    fGeoref(result_ndvi, red_path, result_ndvi_path)
    
    return result_ndvi_path


In [4]:
def fTOA_Refl(band_path, Ap, Mp, out_folder, name = "_TOA_Ref"):   
    
    band= tf.imread(band_path)
    
    result_toa = Mp * band + Ap
    result_toa_path = os.path.join(out_folder, name + ".TIF")
    
    #tf.imsave(result_toa_path , result_toa)
    
    fGeoref(result_toa, band_path, result_toa_path)
    
    return result_toa_path

In [5]:
def fTOA_Rad(thermo_path, ML, AL, out_folder, name = "TOA_Rad"):   
        
    thermo= tf.imread(thermo_path)
    
    result_toa_rad = (ML  * thermo) + AL
    result_toa_rad_path = os.path.join(out_folder, name + ".TIF")
    
    #tf.imsave(result_toa_rad_path , result_toa_rad)
    
    fGeoref(result_toa_rad, thermo_path, result_toa_rad_path)
    
    return result_toa_rad_path

In [6]:
def fBT(input_path_toa, K1, K2, out_folder, name = "BT"):
    
    
    toa_array = np.array(tf.imread(input_path_toa))
    
    zero_exception = np.seterr(all = "ignore")
        
    result_BT = K2/np.log(((K1/toa_array+1))) - 273.15    
    
    result_bt_path = os.path.join(out_folder, name + ".TIF") 
    
    fGeoref(result_BT, input_path_toa, result_bt_path)
    
    
    return result_bt_path


In [7]:
def fVc(ndvi, out_folder, name = "Vc"):
    ndvi_array = np.array(tf.imread(ndvi))
    result_Vc = np.divide(np.power(ndvi_array,2),0.3)

    result_Vc_path = os.path.join(out_folder, name +".TIF")

    fGeoref(result_Vc, ndvi, result_Vc_path)

    return result_Vc_path

In [9]:
def fEmis(vc, ndvi, red, out_folder, name = "Emis"):
        
    ndvi_array = np.array(tf.imread(ndvi))
    vc_array = np.array(tf.imread(vc))
    red = np.array(tf.imread(red))

    result_e = (0.004 * vc_array) + 0.986
    #print(result_e)
    result_e = np.where(ndvi_array < 0.2, 1 - red, result_e) 
    result_e = np.where(ndvi_array > 0.5, 0.99, result_e)              
    result_e[result_e > 1] = 0.99                           
    result_e[result_e < 0.8] = 0.8
    #print(result_e)

    result_e_path = os.path.join(out_folder, name +".TIF")

    fGeoref(result_e, vc, result_e_path)

    return result_e_path

In [10]:
def fLST(bt, surf_emis, out_folder, name = "LST"):
        bt_array = np.array(tf.imread(bt))
        e_array = np.array(tf.imread(surf_emis))
        
        result_LST = (bt_array / (1 + ((0.0015 * bt_array)/1.4488) * np.log(e_array)))
        
        result_LST_path = os.path.join(out_folder, name + ".TIF")
        
        fGeoref(result_LST, bt, result_LST_path)
        
        return result_LST_path